### Derivada | **Monitoria Cálculo**

In [30]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [31]:
raw = pd.read_csv(r'https://raw.githubusercontent.com/gabrielluizone/CalDifInt/main/Data/Calculus/2004-2021.tsv', sep='\t')
raw.sample(3)

,DATA INICIAL,DATA FINAL,REGIÃO,ESTADO,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,MARGEM MÉDIA REVENDA,COEF DE VARIAÇÃO REVENDA,PREÇO MÉDIO DISTRIBUIÇÃO,DESVIO PADRÃO DISTRIBUIÇÃO,PREÇO MÍNIMO DISTRIBUIÇÃO,PREÇO MÁXIMO DISTRIBUIÇÃO,COEF DE VARIAÇÃO DISTRIBUIÇÃO
34817,2012-01-01,2012-01-07,SUL,RIO GRANDE DO SUL,GLP,701,R$/13Kg,39.589,2.709,30.000,48.000,9.835,0.068,29.754,3.526,17.8637,39,0.119
115286,2020-07-26,2020-08-01,NORDESTE,MARANHAO,ETANOL HIDRATADO,57,R$/l,3.480,0.212,2.999,3.999,0.558,0.061,2.922,0.157,2.8663,3.0886,0.054
70436,2014-11-09,2014-11-15,NORDESTE,PERNAMBUCO,ÓLEO DIESEL,100,R$/l,2.489,0.082,2.330,2.700,0.302,0.033,2.187,0.058,2.1237,2.3295,0.027


#### Seleção Amostral
> A [Agência Nacional do Petróleo, Gás Natural e Biocombustíveis ( ANP )](https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/precos-revenda-e-de-distribuicao-combustiveis/serie-historica-do-levantamento-de-precos)
 divulga semanalmente relatórios sobre os preços do gás, diesel e outros combustíveis utilizados no transporte em todo o país. Esses conjuntos de dados trazem o valor médio por litro, número de postos de gasolina analisados ​​e outras informações agrupadas por regiões e estados do país.
Fonte: asa()

- Somente a Sudeste, no estado de São Paulo
- Somente a Gasolina Comum será analisado
- Todo o período
- Somente as colunas: `DATA INICIAL` & `PREÇO MÉDIO REVENDA`

In [32]:
df = raw.loc[# Vamos selecionar as linhas primeiro
    (raw['REGIÃO'] == 'SUDESTE') &
    (raw['ESTADO'] == 'SAO PAULO') &
    (raw['PRODUTO'] == 'GASOLINA COMUM')
    
    ,# Após a virgula, selecionamos as colunas
    ['DATA INICIAL', 'PREÇO MÉDIO REVENDA']
]
df.sample(3)

,DATA INICIAL,PREÇO MÉDIO REVENDA
57384,2013-03-17,2.779
13977,2005-09-25,2.348
20348,2010-04-18,2.454


#### Arrumar as data para que as funções possam funcionar

In [33]:
df['DATA INICIAL'] = pd.to_datetime(df['DATA INICIAL'])

# Calcula os dias decorridos desde a 1 dia de registro
df['DIAS'] = (df['DATA INICIAL'] - df['DATA INICIAL'].min()).dt.days
df.sample(3)

,DATA INICIAL,PREÇO MÉDIO REVENDA,DIAS
87361,2017-01-01,3.605,4620
63327,2013-12-15,2.824,3507
13356,2005-04-03,2.185,329


#### Calcular a derivada das vendas em relação ao dia *(pois existem saltos de dias)*

In [34]:
df['CRESCIMENTO'] = np.gradient(df['PREÇO MÉDIO REVENDA'], df['DIAS'])
df.sample(3)

,DATA INICIAL,PREÇO MÉDIO REVENDA,DIAS,CRESCIMENTO
20969,2010-09-26,2.439,2331,0.000643
91595,2017-07-16,3.234,4816,0.016214
19754,2009-11-15,2.446,2016,-0.000286


#### Um pequeno exemplo dos resultados

In [35]:
np.gradient(np.array([2, 2, 2, 3, 2, 2, 2, 1]))

array([ 0. ,  0. ,  0.5,  0. , -0.5,  0. , -0.5, -1. ])

#### Um exemplo Puro Python

In [36]:
f = [2, 2, 2, 3, 2, 2, 1]
n = len(f)
dfdx = [0] * n

for i in range(1, n-1): # Para os pontos interiores, usamos a fórmula central
    dfdx[i] = (f[i+1] - f[i-1]) / 2

# Para os pontos extremos, usamos a fórmula de diferença para frente e para trás
dfdx[0] = f[1] - f[0]
dfdx[n-1] = f[n-1] - f[n-2]

dfdx

[0, 0.0, 0.5, 0.0, -0.5, -0.5, -1]

#### Visualizando os resultados
1. Se as linhas continuarem abaixo de 0, sua variação é de descida
2. Se as linhas continuarem acima de 0, sua variação é para cima
3. Se as linhas foram próximas de 0, sua variação pode ser 

In [37]:
# Criar subplots com dois gráficos
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1)

# Criar gráfico de variação do preço médio de revenda
scatter_revenda = go.Scatter(x=df['DATA INICIAL'], y=df['PREÇO MÉDIO REVENDA'], mode='lines+markers', name='Preço Médio')
fig.add_trace(scatter_revenda, row=1, col=1)

# Criar gráfico de taxa de crescimento das vendas
scatter_crescimento = go.Scatter(x=df['DATA INICIAL'], y=df['CRESCIMENTO'], mode='lines+markers', name='Taxa de Crescimento')
fig.add_trace(scatter_crescimento, row=2, col=1)

# Adicionar linha horizontal no valor zero de variação
fig.add_hline(y=0, line_dash="dash", line_color="red", row=2, col=1)

# Configurar layout dos gráficos
fig.update_layout(height=600, width=800, title_text="Variação do Preço Médio e Taxa de Crescimento das Vendas ao Longo do Tempo")
fig.update_xaxes(title_text='Data Inicial')
fig.update_yaxes(title_text='Preço Médio', row=1, col=1)
fig.update_yaxes(title_text='Taxa de Crescimento', row=2, col=1)
fig.update_layout(legend_traceorder='reversed')
fig.show()